# Hello World Pipeline

## Section 1: Introduction

The goal of this notebook is to show how you can turn a notebook into an API.

## Section 2: Pipeline API

In [ ]:
# pipeline-api
from unstructured_inference.inference.layout import process_data_with_model
from unstructured_inference.models.base import UnknownModelException
from fastapi import HTTPException

In [ ]:
from fastapi import status,Form
from typing import List

In [ ]:
# pipeline-api
ALL_ELEMS = "_ALL"
VALID_FILETYPES = ["application/pdf", "image/png"]

def pipeline_api(
    file,
    file_content_type=None,
    m_model_type = [],
    m_force_ocr=[],
    m_include_elems= [],

):
    if file_content_type not in VALID_FILETYPES:
        raise HTTPException(status.HTTP_404_NOT_FOUND)

    is_image = file_content_type == "image/png"
    model = None  if m_model_type == [] else m_model_type[0]
    ocr_strategy = "force" if m_force_ocr else "auto"
    try:
        layout = process_data_with_model(
            file, model, is_image, ocr_strategy=ocr_strategy
        )  # type: ignore
    except UnknownModelException as e:
        raise HTTPException(status.HTTP_422_UNPROCESSABLE_ENTITY, str(e))
    pages_layout = [
        {
            "number": page.number,
            "elements": [
                element.to_dict()
                for element in page.elements
                if element.type in m_include_elems or m_include_elems == ALL_ELEMS
            ],
        }
        for page in layout.pages
    ]

    return {"pages": pages_layout}